In [18]:
import pandas as pd
import unicodedata
import numpy as np
import os

In [19]:


def leer_nombres_archivos():
    """Obtiene una lista de archivos en el directorio actual"""
    archivos = [archivo for archivo in os.listdir(os.getcwd())]
    return archivos

def tratar_texto(texto):
    """Normaliza cadenas eliminando tildes y caracteres especiales"""
    if isinstance(texto, str):
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
        return texto_sin_tildes
    elif isinstance(texto, pd.Timestamp):
        return texto.strftime("%d-%m-%Y")
    else:
        return texto  # Si no es texto, devuelve el valor original

def diccionario_archivos_excel():
    """Crea un diccionario con los datos de los archivos Excel procesados"""
    archivos_excel = {}
    for archivo in leer_nombres_archivos():
        # Solo procesar archivos con extensión .xlsx
        if archivo.endswith(".xlsx"):
            ruta_completa = os.path.join(os.getcwd(), archivo)
            try:
                # Cargar el archivo Excel
                df = pd.read_excel(
                    ruta_completa,
                    skiprows=7,
                    usecols="D:K",
                    engine="openpyxl"
                )
                # Renombrar columnas específicas
                df = df.rename(columns={
                    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
                    "FECHA POSESIÓN": "FECHA_POSESION",
                    "FECHA BAJA": "FECHA_BAJA",
                    "CÓDIGO INE": "CODIGO_NIE"
                })
                # Aplicar tratamiento de texto a cada celda
                df_sin_tildes = df.map(tratar_texto)
                # Guardar el DataFrame en el diccionario
                nombre_base = os.path.splitext(archivo)[0]
                archivos_excel[nombre_base] = df_sin_tildes
                print(f"Datos del archivo {archivo} cargados con éxito.")
            except Exception as e:
                print(f"Error al procesar el archivo {archivo}: {e}")
    return archivos_excel

# Procesar los archivos Excel
archivos_excel = diccionario_archivos_excel()

# Guardar los resultados en un único archivo Excel
archivo_salida = "prueba.xlsx"
with pd.ExcelWriter(archivo_salida, engine="openpyxl") as writer:
    for nombre_hoja, df in archivos_excel.items():
        # Guardar cada DataFrame en una hoja separada
        df.to_excel(writer, sheet_name=nombre_hoja, index=False)

print(f"Archivo combinado guardado como {archivo_salida}")


Datos del archivo Alcaldes_Mandato_1979_1983.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1983_1987.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1987_1991.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1991_1995.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1995_1999.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1999_2003.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2003_2007.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2007_2011.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2011_2015.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2015_2019.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2019_2023.xlsx cargados con éxito.
Archivo combinado guardado como prueba.xlsx


In [20]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
    
datos = pd.concat(hojas.values(), ignore_index=True)

1.En esta Burbuja vamos a saber que alcalde repetio el mandanto y en que lexislaturas estuvo.

In [21]:
df_repetidos = df.groupby(["NOMBRE"]).size().reset_index(name="Veces_Elegido")
df_repetidos =  df_repetidos[df_repetidos["Veces_Elegido"] > 1].sort_values("Veces_Elegido")
print(df_repetidos)

                             NOMBRE  Veces_Elegido
1266         CARLOS MARTIN GONZALEZ              2
2411   FRANCISCO JAVIER ABAD GARCIA              2
3275           JAVIER GARCIA GARCIA              2
4024       JOSE FERNANDEZ FERNANDEZ              2
4396       JOSE MARIA PEREZ SANCHEZ              2
4413    JOSE MARIA VIJUESCA LEDESMA              2
4438        JOSE MIGUEL GOMEZ GOMEZ              2
5823            MANUEL MARTIN PEREZ              2
6149  MARIA DEL CARMEN MORENO PEREZ              2
7888           ROBERTO MUNOZ GARCIA              2
7896    ROBERTO RODRIGUEZ RODRIGUEZ              2
8008           RUBEN GOMEZ GONZALEZ              2


2.En esta burbuja vamos a sacar los partidos con mas exito, para eso vamos a combinar todas las hojas en un solo dataFrame para obtener todos los datos de todas las hojas que tenemos.

In [22]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
    
datos = pd.concat(hojas.values(), ignore_index=True)
df_partidos_exitosos = df.groupby(["LISTA"]).size().reset_index(name="Veces_Totales")
df_partidos_exitosos = df_partidos_exitosos.sort_values("Veces_Totales", ascending=False)
print(df_partidos_exitosos.head(5))

           LISTA  Veces_Totales
25            PP           3029
29          PSOE           2914
4   C. ELECTORAL            993
21         OTROS            558
17           IND            283


2.En esta burbuja vamos a comprobar los alcaldes que cambiaron de partido y tambien combinar todas las hojas en un solo DataFrame.

In [23]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
datos = pd.concat(hojas.values(), ignore_index=True)

df_alcaldes_veletas = df.groupby("NOMBRE")["LISTA"].nunique().reset_index(name="Veces_Totales")
df_alcaldes_veletas = df_alcaldes_veletas[df_alcaldes_veletas["Veces_Totales"] > 1]
print(df_alcaldes_veletas)

                             NOMBRE  Veces_Totales
4396       JOSE MARIA PEREZ SANCHEZ              2
4438        JOSE MIGUEL GOMEZ GOMEZ              2
6149  MARIA DEL CARMEN MORENO PEREZ              2
7888           ROBERTO MUNOZ GARCIA              2
7896    ROBERTO RODRIGUEZ RODRIGUEZ              2
8008           RUBEN GOMEZ GONZALEZ              2


3.En esta Burbuja vamos a calcular los alcades que cesaron antes de tiempo.

In [24]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
datos = pd.concat(hojas.values(), ignore_index=True)

df_alcaldes_caidos = datos[datos["FECHA_BAJA"].notnull()] #datos.groupby("NOMBRE").size().reset_index()
df_alcaldes_caidos1 = df_alcaldes_caidos.groupby(["NOMBRE", "LISTA", "PROVINCIA", "COMUNIDAD_AUTONOMA"]).size()
df_alcaldes_caidos2 = df_alcaldes_caidos.groupby(["NOMBRE", "LISTA"]).size().reset_index(name="Veces_Totales")
df_alcaldes_caidos2 = df_alcaldes_caidos2[df_alcaldes_caidos2["Veces_Totales"] > 1]
print(df_alcaldes_caidos2)

                         NOMBRE LISTA  Veces_Totales
25     ADOLFO CAMPOS PANADEIROS    PP              2
51        AGUSTIN BAAMONDE DIAZ    PP              2
117    ALEJANDRO CASTRO BERMEJO    PP              2
256        AMPARO NUNEZ SANCHEZ    PP              2
302   ANDRES SAMPERIO MONTEJANO    PP              2
...                         ...   ...            ...
6147  VALLE GARMENDIA, MAXIMINO  PSOE              2
6205     VERONICA GOMEZ GALLEGO  PSOE              2
6211      VICENTA BOSCH PALANCA  PSOE              2
6233     VICENTE LLANOS VAZQUEZ  PSOE              2
6312   VINARAS MUNOZ, FRANCISCO  PSOE              2

[93 rows x 3 columns]


4.En esta Burbuja vamos a calcular cual es el sueldo mas alto y el alcalde correspondiente.

In [36]:
ruta_archivo_alcaldes = "Alcaldes_Mandato_2019_2023.xlsx"
ruta_archivo_retribuciones = "tabula-retribuciones_alcaldes_2021.csv"
df_alcaldes = pd.read_excel(
    ruta_archivo_alcaldes,
    skiprows=7,
    usecols="D:K",
    engine="openpyxl"
)
df_alcaldes = df_alcaldes.rename(columns={
    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
    "FECHA POSESIÓN": "FECHA_POSESION",
    "FECHA BAJA": "FECHA_BAJA",
    "CÓDIGO INE": "CODIGO_NIE"
})
df_alcaldes = df_alcaldes.map(tratar_texto)
df_retribuciones = pd.read_csv(ruta_archivo_retribuciones, skiprows=1)
df_retribuciones = df_retribuciones.rename(columns={"CCAA": "COMUNIDAD_AUTONOMA",
                                                    "DEDICACIÓN": "REGIMEN_DEDICACION",
                                                    "PERCIBIDO": "TOTAL_PERCIBIDO",
                                                    "AYUNTAMIENTO": "MUNICIPIO"
                                                    }
                                           )
df_retribuciones = df_retribuciones.map(tratar_texto)
df_retribuciones_alcaldes = pd.merge(df_alcaldes, df_retribuciones, on="MUNICIPIO", how="inner")

# Formatear retribuciones como números
df_retribuciones_alcaldes["TOTAL_PERCIBIDO"] = df_retribuciones_alcaldes["TOTAL_PERCIBIDO"].str.replace(",", "").astype(float)


# Ordenar por mayor retribución
df_retribuciones_alcaldes = df_retribuciones_alcaldes.sort_values("TOTAL_PERCIBIDO", ascending=False)
df_retribuciones_alcaldes


,COMUNIDAD_AUTONOMA_x,PROVINCIA_x,MUNICIPIO,CODIGO_NIE,NOMBRE,LISTA,FECHA_POSESION,FECHA_BAJA,PROVINCIA_y,COMUNIDAD_AUTONOMA_y,REGIMEN_DEDICACION,TOTAL_PERCIBIDO
4056,Castilla-La Mancha,Guadalajara,Horche,191431,JUAN MANUEL MORAL CALVETE,PSOE,15/06/2019,NaN,Guadalajara,Castilla-La Mancha,Exclusiva,4379787.0
3297,Castilla y Leon,Valladolid,"Mudarra, La",470992,VICTOR GONZALEZ GARABITO,PP,15/06/2019,NaN,Valladolid,Castilla y Leon,Sin dedicacion,99000.0
4432,Cataluna,Barcelona,Pujalt,81764,PERE MASANA NADAL,C. ELECTORAL,15/06/2019,NaN,Barcelona,Cataluna,Sin dedicacion,99000.0
2199,Castilla y Leon,Leon,Destriana,240665,ABELARDO FELGUEROSO SANTAMARINA,PP,15/06/2019,NaN,Leon,Castilla y Leon,Sin dedicacion,98000.0
5898,Extremadura,Caceres,Navezuelas,101344,CARLOS JAVIER RIOS PEROMINGO,PP,15/06/2019,NaN,Caceres,Extremadura,Sin dedicacion,97000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7064,"Rioja, La","Rioja, La",Ventosa,261639,Ricardo Velasco Garcia,PSOE,15/06/2019,NaN,"Rioja, La","Rioja, La",Sin dedicacion,0.0
7065,"Rioja, La","Rioja, La",Ventrosa,261644,Carlos Fernandez Rueda,PP,15/06/2019,NaN,"Rioja, La","Rioja, La",Sin dedicacion,0.0
7066,"Rioja, La","Rioja, La",Viguera,261657,Alvaro Manzanos Santibanez,PP,15/06/2019,NaN,"Rioja, La","Rioja, La",Sin dedicacion,0.0
7083,"Rioja, La","Rioja, La",Zarzosa,261814,Jose Maria Saenz Blanco,PP,15/06/2019,NaN,"Rioja, La","Rioja, La",Sin dedicacion,0.0


5.En esta Burbuja vamos a relacionar los sueldos altos y bajos con la probalidad de repetir mandato.

Sueldo_Clasificacion
Alto    0.906886
Bajo    0.949437
Name: REPETIR_MANDATO, dtype: float64


6.En esta Burbuja vamos a hacer otra Información